In [31]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('NYC_Dog_Licensing_Dataset_.csv')

/var/folders/v2/9k8drrwj5h39kpr935vvx6j80000gn/T/ipykernel_26394/3073573986.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('NYC_Dog_Licensing_Dataset_.csv')


In [3]:
df.dtypes

AnimalName             object
AnimalGender           object
AnimalBirthYear        object
BreedName              object
ZipCode               float64
LicenseIssuedDate      object
LicenseExpiredDate     object
Extract Year            int64
dtype: object

In [4]:
print("Total rows:", len(df))

Total rows: 722864


In [5]:
df.head(100)

,AnimalName,AnimalGender,AnimalBirthYear,BreedName,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,10035.0,09/12/2014,09/12/2017,2016
1,YOGI,M,2010,Boxer,10465.0,09/12/2014,10/02/2017,2016
2,ALI,M,2014,Basenji,10013.0,09/12/2014,09/12/2019,2016
3,QUEEN,F,2013,Akita Crossbreed,10013.0,09/12/2014,09/12/2017,2016
4,LOLA,F,2009,Maltese,10028.0,09/12/2014,10/09/2017,2016
...,...,...,...,...,...,...,...,...
95,SPUDS,M,2010,Bulldog,10461.0,09/17/2014,09/17/2017,2016
96,BOO,F,2013,Labradoodle,11211.0,09/17/2014,09/17/2019,2016
97,SAMMY,M,2006,Shiba Inu,10011.0,09/17/2014,10/31/2016,2016
98,MAX,M,2005,Pug,10025.0,09/17/2014,10/30/2016,2016


In [6]:
df.keys()

Index(['AnimalName', 'AnimalGender', 'AnimalBirthYear', 'BreedName', 'ZipCode',
       'LicenseIssuedDate', 'LicenseExpiredDate', 'Extract Year'],
      dtype='object')

### 🛠️ Step 1: Fix the Data Types for Each Column

Ensure all columns have the correct data types to support accurate analysis:
- Convert dates (e.g., `LicenseIssuedDate`) to `datetime`
- Convert numeric values (e.g., `AnimalBirthYear`) to `int`
- Convert `ZipCode` to string to preserve leading zeros and allow proper grouping


In [7]:
# Convert dates to datetime
df['LicenseIssuedDate'] = pd.to_datetime(df['LicenseIssuedDate'], errors='coerce')
df['LicenseExpiredDate'] = pd.to_datetime(df['LicenseExpiredDate'], errors='coerce')


In [8]:
# Convert birth year to numeric
df['AnimalBirthYear'] = pd.to_numeric(df['AnimalBirthYear'], errors='coerce')


In [9]:
# Convert zip code to string so we can group by it without decimal places
df['ZipCode'] = df['ZipCode'].astype('Int64').astype(str)

In [10]:
df['BreedName'] = df['BreedName'].str.strip()
df['AnimalGender'] = df['AnimalGender'].str.strip()


In [11]:
df.dtypes

AnimalName                    object
AnimalGender                  object
AnimalBirthYear              float64
BreedName                     object
ZipCode                       object
LicenseIssuedDate     datetime64[ns]
LicenseExpiredDate    datetime64[ns]
Extract Year                   int64
dtype: object

### Step 2: Check for Full Row Duplicates

Instead of checking for duplicates column by column (which might drop valid but similar entries like dogs with the same name), we checked for **exact duplicate rows**. 

For example, there might be two dogs named Luna, but they could have different birth years, breeds, or license dates, so we **only dropped rows that were completely identical across all columns**, ensuring no meaningful data was lost.


In [12]:
df.duplicated().sum()

np.int64(40545)

In [13]:
df = df.drop_duplicates()

In [14]:
df.duplicated().sum()

np.int64(0)

In [15]:
print("Total rows:", len(df))


Total rows: 682319


### 🐾 Step 3: Create Age Groups for Dogs

We used the `ApproxDogAge` column to categorize dogs into meaningful life stages. This helps support business questions about age distribution and trends.

We defined age bins as follows:
- `0–1` → Puppy  
- `2–3` → Young  
- `4–6` → Adult  
- `7–10` → Senior  
- `10+` → Elderly  

We also ensured proper display by replacing any special en dashes (–) with regular hyphens (-) to avoid formatting issues in Excel.


In [32]:
df['LicenseIssuedYear'] = df['LicenseIssuedDate'].dt.year
# Replace negative or unrealistic ages with NaN
df['ApproxDogAge'] = df['ApproxDogAge'].apply(lambda x: x if 0 <= x <= 25 else np.nan)


In [33]:
print(df['ApproxDogAge'].dtype)  # Should be float or int


float64


In [34]:
df['DogAgeGroup'] = pd.cut(
    df['ApproxDogAge'],
    bins=[0, 1, 3, 6, 10, 20],
    labels=['0-1 (Puppy)', '2-3 (Young)', '4-6 (Adult)', '7-10 (Senior)', '10+ (Elderly)'],
    include_lowest=True
)
# Clean encoding for Excel display
df['DogAgeGroup'] = df['DogAgeGroup'].str.replace('-', '-', regex=False)


### Step 4: Identify and Handle Missing Values

We checked for missing data across all columns to understand where information was incomplete.

Key findings:
- `AnimalName` had 1,709 missing entries — we decided to keep these since a name doesn't affect license validity.
- `AnimalGender`, `AnimalBirthYear`, and `LicenseExpiredDate` had fewer missing values but were important for analysis.

To maintain data quality:
- We dropped rows where **any of these 3 key columns** were missing:  
  `AnimalBirthYear`, `LicenseExpiredDate`, `AnimalGender`.


In [35]:
df.isnull().sum()

AnimalName              0
AnimalGender            0
AnimalBirthYear         0
BreedName               0
ZipCode                 0
LicenseIssuedDate       0
LicenseExpiredDate      0
Extract Year            0
ApproxDogAge           97
DogAgeGroup           343
IssueYear               0
ExpireYear              0
IsBadName               0
CleanedBreedName        0
LicenseIssuedYear       0
dtype: int64

In [36]:
df[df[['AnimalBirthYear', 'LicenseExpiredDate', 'AnimalGender']].isnull().any(axis=1)].shape[0]


0

This step removed 130 rows, ensuring our data remains reliable for further analysis.


In [37]:
# Keep only rows where all three of these values are filled in. Otherwise, lose 130 rows! 
df = df.dropna(subset=['AnimalBirthYear', 'LicenseExpiredDate', 'AnimalGender'])


In [38]:
print("Total rows:", len(df))


Total rows: 546717


### Step 5: Clean Invalid ZIP Codes

To ensure accuracy in geographic analysis, we filtered the ZIP codes so that only valid entries remain.

- A valid NYC ZIP code must be **5 digits** long.
- Some rows had invalid or missing ZIP codes (like `0`, `100`, or empty), which we removed.


In [39]:
df['ZipCode'] = df['ZipCode'].astype(str)
df = df[df['ZipCode'].str.len() == 5] 

In [40]:
print("Total rows:", len(df))


Total rows: 546717


### Step 6: Validate Extract Year

We found rows where the `Extract Year` didn't make logical sense. For example, if a dog's license expired in 2021 but the extract year was 2022, that implies the license was still active — which can't be true.

✅ Our fix:
- We ensured the extract year falls between the license issue and license expiration years


In [41]:
# Extract the year from issued and expired dates
df['IssueYear'] = df['LicenseIssuedDate'].dt.year
df['ExpireYear'] = df['LicenseExpiredDate'].dt.year

# Keep only rows where extract year is between issue and expire years
df = df[
    (df['Extract Year'] >= df['IssueYear']) &
    (df['Extract Year'] <= df['ExpireYear'])
]


In [42]:
#its redundant
df = df.drop(columns=['LicenseIssuedYear'])

In [43]:
zipcodes = pd.read_csv('zipcode.csv')
df['ZipCode'] = df['ZipCode'].astype(str).str.strip().str.replace(r'\.0$', '',regex=True)
valid_zips = zipcodes['NYC Zipcodes'].astype(str).str.strip().str.replace(r'\.0$', '', regex = True)
df = df[df['ZipCode'].isin(valid_zips)]

We wanted to identify how many "bad" or placeholder dog names exist in the dataset. While we are not deleting them, since they aren’t directly related to our business question, we aimed to avoid unnecessary data loss and maintain as much usable information as possible.



In [44]:
# no name 
df['AnimalName'] = df['AnimalName'].str.upper()
#a list of names that are bad
bad_names = {
        'NAME', 'NAME NOT PROVIDED', 'NAME?', 'NAMEME',
    'NO NAME', 'NONAME', 'UNNAME', 'WHATSHERNAME', '.', '2'
}

df['AnimalName'] = df['AnimalName'].fillna('').str.strip()

df['IsBadName'] = df['AnimalName'].isin(bad_names)

# Count total unusable and usable names
bad_name_count = df['IsBadName'].sum()
usable_name_count = (~df['IsBadName']).sum()

print(f"Unusable dog names: {bad_name_count}")
print(f"Usable dog names: {usable_name_count}")



Unusable dog names: 9099
Usable dog names: 537618


In [45]:
import re

# Replace all variations with the word "Mix"
df['BreedName'] = (
    df['BreedName']
    .str.replace(r'\b(Mix|Crossbreed|Cross|X)\b', 'Mix', regex=True)  # standardize
    .str.replace(r'\s+', ' ', regex=True)  # remove double spaces
    .str.strip()  # trim leading/trailing space
    .str.title()  # format like "Poodle Mix"
)


In [46]:
from thefuzz import process

# unique_breeds = df['BreedName']
unique_breeds = df['BreedName'].dropna().astype(str).unique()
# choose a reference list of "cleaned" breeds
reference_breeds = [
    'Airedale Terrier', 'Airedale Terrier Mix',
    'American Eskimo Mix', 'American Ekimo',
    'American Pit Bull Terrier Mix',
    'American Staffordshie Terrier','American Staffordshie Terrier Mix',
    'Australian cattle dog', 'Basset Hound', 'Australian Shepherd', 'Berger Picard', 
    'Bichon Poodle', 'Bichon Shih Tzu', 'Bichon Frise', 'Yorkie Bichon', 'Catahoula leopard',
    'Cavalier King Charles Spaniel', 'Chihuahua', 'Chihuahua Mix', 'long-haired Chihuahua', 'Border Collie',
    'Coton de Tulear','Cockapoo', 'Toy Australian Shepherd', 'Dachshund smooth coat', 'Golden Retrievers', 'Goldendoodle', 'Miniature Goldendoodles',
    'chinese crested', 'German Shepherd', 'Corgi German Shepherd', 'Husky German Shepherd', 'jack russell chihuahua', 'labrador retriever', 'Yorkie Poodle',
    'Spanish Water Dog', 'small munsterlander', 'Shorkie', 'Shitzu Poodle', 'Pitbull pointer', 'Teacup Poodle', 'Pekapoo', 'jack russell',
    'Lhasapoo', 'Maltipoo', 'mini pinscher', 'Mixed', 'mini labradoodle', 'Westiepoo', 'French Poodle', 'Peek-a-boo', 'Shorkie', 'labrador cattle dog', 'Taiwan Mountain',
    'cockapoo', 'Cock-a-poo', 'yorkie maltese'
]
breed_map = {}
for breed in unique_breeds:
    match, score = process.extractOne(breed, reference_breeds)
    if score >= 85:  # Match threshold
        breed_map[breed] = match

# Step 5: Apply cleaned breed names — fallback to original if no match
df['CleanedBreedName'] = df['BreedName'].apply(lambda x: breed_map.get(x, x))

# Step 6 (optional): Check unmatched entries
unmatched_breeds = set(unique_breeds) - set(breed_map.keys())
print(f"\n✅ Total unique breeds: {len(unique_breeds)}")
print(f"🟡 Matched breeds: {len(breed_map)}")
print(f"❌ Unmatched breeds (still messy): {len(unmatched_breeds)}")
print(unmatched_breeds)



✅ Total unique breeds: 1125
🟡 Matched breeds: 689
❌ Unmatched breeds (still messy): 436
{'Pompoo', 'Hound Mastiff', 'Morky', 'Yellow Lab', 'Westie-Poodle', 'Not Provided', 'Lhasa Apso/Shih', 'Leonberger', 'Maltes/Shih Tzu', 'Schnauzer/Poodle', 'Lhasa-Bijon', 'Bichon/Cairn', 'Chintese', 'Hunting Dog/Rescue', 'Shep/Doxie', 'Engish Pointer', 'Samoyed', 'Japanese Chin/Spaniel', 'Miniature Schnauzer', 'Maltese/Pomeranian', 'Finnish Spitz', 'Poodle/Pompoo', 'Smooth Hair Fox Terrier', 'Sapsal, Korea', 'Mastiff', 'Eng Setter', 'Mountain Feist', 'English Setter', 'Poodle, Miniature', 'Irish Wolfhound', 'English Springer Spaniel', 'Jap Chin / Maltese', 'Rhodesian Ridgeback', 'Labrador/G.Shep', 'Whippet', 'Belgian Griffon', 'Canaan', 'Peke-A-Poo', 'Shihtzu-A-Poo', 'Scott Terr/Shih Tsu', 'Pomtese', 'Mastiff, Tibetan', 'Newfoundland', 'Schipperkee', 'Coonhound, Redbone', 'St. Bernard', 'Blue Heeler / Schnauser', 'Ridgeback/Rottweiler', 'Peke A Ppp', 'Cokaspinial Poodle', 'Japanese Spitz', 'Standar

In [47]:
df.to_csv('Cleaned_NYC_Dog_License.csv', index=False, encoding='utf-8-sig')


In [48]:
len(df)

546717